# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.data.dataset_factory import TabularDatasetFactory
from train import clean_data
from azureml.train.automl import AutoMLConfig
from azureml.automl.runtime.onnx_convert import OnnxConverter
from azureml.widgets import RunDetails
from pprint import pprint
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import Model
import requests

['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']


/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook134535/code/Users/odl_user_134535/nd00333-capstone-master/nd00333-capstone-master/train.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_red["wine_type"] = "RED"


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'wine-quality-automl'

experiment=Experiment(ws, experiment_name)

# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "wine-quality"
description_text = "Wine Quality Dataset for Udacity Course 3"

if key in ws.datasets.keys(): 
    found = True
    input_data = ws.datasets[key] 

if not found:
    # Create AML Dataset and register it into Workspace
    url_white = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv"
    url_red = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    white_data = TabularDatasetFactory.from_delimited_files(url_white, separator=";")
    red_data = TabularDatasetFactory.from_delimited_files(url_red, separator=";")
    features, target = clean_data(white_data, red_data, numeric_target=False)
    features["quality"]=target
    ds = ws.get_default_datastore()
    input_data = TabularDatasetFactory.register_pandas_dataframe(dataframe=features, target=ds, name=key,
                                                             description=description_text)


df = input_data.to_pandas_dataframe()
df.describe()


In [ ]:
# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "aml-compute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D12_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
automl_settings = {
    "n_cross_validations": 3,
    "enable_early_stopping": True,
    "primary_metric": "AUC_weighted",
    "max_concurrent_iterations": 4,
    
}
automl_config = AutoMLConfig(
    experiment_timeout_hours=1.0,
    task="classification",
    compute_target=compute_target,
    training_data=input_data,
    label_column_name="quality",
    **automl_settings)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, tag={'type':'automl'}, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
automl_run.wait_for_completion()
assert (automl_run.get_status() == "Completed")
best_ml_run, best_model = automl_run.get_output()


def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators':
                       list(e[0] for e in step[1].estimators),
                   'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()
            
print_model(best_model)


In [ ]:
best_ml_run.download_file("./outputs/model.pkl")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
registered_model = best_model.register(model_name="wine-quality-model", model_path="model.pkl")
environment = Environment('my-environment')
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults==1.20.0',
    'inference-schema[numpy-support]==1.1.0',
    'joblib==1.0.0',
    'numpy==1.19.5',
    'scikit-learn==0.24.0'
])
environment.save_to_directory(path='env_automl.yml')
service_name = 'wine-quality-classification'
inference_config = InferenceConfig(entry_script='score.py', environment=environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)
primary_key, _ = service.get_keys()

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
data = {"data":
         [{
             "is_red_wine": 1,
             "fixed_acidity":7.90000,
             "volatile acidity":0.52000,
             "citric acid":0.26000, 
             "residual sugar":2.2000,
             "chlorides":0.07900
             "free sulfur dioxide":14.0000,
             "total sulfur dioxide":38.0000,
             "density":0.99675,
             "pH":3.31000,
             "sulphates":0.62000,
             "alcohol":10.20000
         },
         {
             "is_red_wine": 0,
             "fixed_acidity":6.80,
             "volatile acidity":0.26000,
             "citric acid":0.32000, 
             "residual sugar":5.2000,
             "chlorides":0.04300
             "free sulfur dioxide":34.0000,
             "total sulfur dioxide":134.00000,
             "density":0.99374,
             "pH":3.18000,
             "sulphates":0.47000,
             "alcohol":10.40000
         }]
     }
header = {'Content-Type': 'application/json'}
header["Authorization"] = f"Bearer {primary}"
json_data = json.dumps(data)

resp = requests.post(service.scoring_uri, json_data, headers=header)
print(resp)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())
#service.delete()